In [3]:
import os
# change the OS to use your project folder as the working directory
project_folder = "/content/drive/MyDrive/Colab_Notebooks/depression_detection/"
os.chdir(project_folder)

In [4]:
import pandas as pd
final_text=[]
for file in os.listdir(project_folder):
  if file.endswith("_TRANSCRIPT.csv"):
    data_features = pd.read_csv(file,delimiter='\t')
    data_values= data_features['value']
    value_list = data_values.tolist()
    final_text.append(value_list)

In [5]:
import pandas as pd

train_labels = pd.read_csv('train_split_Depression_AVEC2017.csv')
dev_labels = pd.read_csv('dev_split_Depression_AVEC2017.csv')
test_labels = pd.read_csv('full_test_split.csv')


In [6]:
test_labels=test_labels.rename(columns={"PHQ_Binary": "PHQ8_Binary"})

In [7]:
# If not, please adjust the column names accordingly
combined_data = pd.concat([train_labels, dev_labels])
combined_data = pd.concat([combined_data, test_labels])

In [8]:
combined_data=combined_data.sort_values(by=['Participant_ID'])

In [ ]:
len(final_text[0])

240

### **CHATGPT 4**


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer, recall_score, f1_score, confusion_matrix


# Extract the 'PHQ8_Binary' label
labels = combined_data['PHQ8_Binary'].tolist()

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import make_scorer, recall_score, confusion_matrix

# Custom scorer for specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)


# Flatten each inner list to a single string
flattened_text = [' '.join(map(str, doc)) for doc in final_text]


# Define the pipeline
pipeline = make_pipeline(
    CountVectorizer(),  # Convert text data to numeric features
    LogisticRegression(max_iter=1000)  # Apply a logistic regression model
)

# Define scorers
scoring = {
    'accuracy':'accuracy',
    'f1_score': 'f1',  # Directly use 'f1' string shortcut for f1 score
    'sensitivity': make_scorer(recall_score),  # recall is the same as sensitivity
    'specificity': make_scorer(specificity_score)
}

# Perform 5-fold cross-validation using the pipeline and text data
results = cross_validate(pipeline, flattened_text, labels, cv=5, scoring=scoring, return_train_score=False)

# Calculate mean and standard deviation for F1, sensitivity, and specificity
acc_mean=np.mean(results['test_accuracy'])
acc_std=np.std(results['test_accuracy'])
f1_mean = np.mean(results['test_f1_score'])
f1_std = np.std(results['test_f1_score'])
sensitivity_mean = np.mean(results['test_sensitivity'])
sensitivity_std = np.std(results['test_sensitivity'])
specificity_mean = np.mean(results['test_specificity'])
specificity_std = np.std(results['test_specificity'])

# Print results
print("Accuracy - Mean: {:.3f}, Standard Deviation: {:.3f}".format(acc_mean, acc_std))
print("F1 Score - Mean: {:.3f}, Standard Deviation: {:.3f}".format(f1_mean, f1_std))
print("Sensitivity - Mean: {:.3f}, Standard Deviation: {:.3f}".format(sensitivity_mean, sensitivity_std))
print("Specificity - Mean: {:.3f}, Standard Deviation: {:.3f}".format(specificity_mean, specificity_std))


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, recall_score, confusion_matrix
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')


# Symptom keywords and their associated weights

# Mapping of checklist items to keywords
symptom_keywords = {
    "sad": 3, "depressed": 3, "down": 2,
    "unhappy": 2, "blue": 2, "miserable": 2,
    "crying": 2, "tearful": 2, "weeping": 2,
    "discouraged": 2, "disheartened": 2, "hopeless": 3,
    "worthless": 3, "inadequate": 3, "inferior": 2,
    "guilty": 2, "ashamed": 2, "remorse": 1,
    "self-criticism": 1, "blaming": 1,
    "indecisive": 1, "hesitant": 1, "uncertain": 1,
    "disinterest in relationships": 2, "social withdrawal": 2, "isolation": 2,
    "lonely": 2, "isolated": 2, "alone": 2,
    "avoiding social interactions": 2, "withdrawal": 2, "seclusion": 2,
    "unmotivated": 2, "apathetic": 2, "listless": 2,
    "disinterest in work": 2, "bored": 2, "apathetic about activities": 2,
    "avoiding responsibilities": 2, "neglecting duties": 2, "shirking": 1,
    "anhedonia": 3, "joyless": 3, "pleasureless": 3,
    "fatigue": 1, "exhausted": 1, "worn out": 1,
    "insomnia": 1, "oversleeping": 1, "sleep disturbances": 1,
    "eating less": 1, "eating more": 1, "appetite changes": 1,
    "low libido": 1, "disinterest in sex": 1, "sexual apathy": 1,
    "health anxiety": 1, "hypochondria": 1, "preoccupied with health": 1
}


# Function to analyze text and generate a rating based on symptom keywords
def generate_rating(text):
    words = word_tokenize(text.lower())
    return sum(symptom_keywords.get(word, 0) for word in words)

# Flatten each inner list to a single string
flattened_text = [' '.join(map(str, doc)) for doc in final_text]
ratings = [generate_rating(text) for text in flattened_text]

# Convert text data to numeric features using CountVectorizer within a pipeline
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(flattened_text).toarray()

# Append ratings to feature matrix
ratings_array = np.array(ratings).reshape(-1, 1)
X_combined = np.hstack((X_text, ratings_array))

# Extract the 'PHQ8_Binary' label
labels = combined_data['PHQ8_Binary'].tolist()


# Define the pipeline with LogisticRegression
pipeline = make_pipeline(
    LogisticRegression(max_iter=1000)
)

# Define custom scorers
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

scoring = {
    'accuracy':'accuracy',
    'f1_score': 'f1',
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score)
}

# Perform 5-fold cross-validation using the pipeline and combined data
results = cross_validate(pipeline, X_combined, labels, cv=5, scoring=scoring, return_train_score=False,  error_score='raise')

# Calculate mean and standard deviation for F1, sensitivity, and specificity
acc_mean=np.mean(results['test_accuracy'])
acc_std=np.std(results['test_accuracy'])
f1_mean = np.mean(results['test_f1_score'])
f1_std = np.std(results['test_f1_score'])
sensitivity_mean = np.mean(results['test_sensitivity'])
sensitivity_std = np.std(results['test_sensitivity'])
specificity_mean = np.mean(results['test_specificity'])
specificity_std = np.std(results['test_specificity'])

# Print results
print("Accuracy - Mean: {:.3f}, Standard Deviation: {:.3f}".format(acc_mean, acc_std))
print("F1 Score - Mean: {:.3f}, Standard Deviation: {:.3f}".format(f1_mean, f1_std))
print("Sensitivity - Mean: {:.3f}, Standard Deviation: {:.3f}".format(sensitivity_mean, sensitivity_std))
print("Specificity - Mean: {:.3f}, Standard Deviation: {:.3f}".format(specificity_mean, specificity_std))


In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score

nltk.download('punkt')

# Define symptom keywords with adjusted weights and expanded coverage
symptom_keywords = {
    "sad": 3, "depressed": 4, "down": 3, "unhappy": 2, "blue": 2, "miserable": 3,
    "crying": 3, "tearful": 2, "weeping": 2, "discouraged": 3, "disheartened": 2, "hopeless": 4,
    "worthless": 4, "inadequate": 3, "inferior": 2, "guilty": 2, "ashamed": 2, "remorse": 2,
    "self-criticism": 2, "blaming": 2, "indecisive": 1, "hesitant": 1, "uncertain": 1,
    "disinterest in relationships": 3, "social withdrawal": 3, "isolation": 3,
    "lonely": 3, "isolated": 2, "alone": 2, "avoiding social interactions": 3, "withdrawal": 2, "seclusion": 2,
    "unmotivated": 3, "apathetic": 3, "listless": 2, "disinterest in work": 2, "bored": 2, "apathetic about activities": 2,
    "avoiding responsibilities": 3, "neglecting duties": 2, "shirking": 1,
    "anhedonia": 4, "joyless": 3, "pleasureless": 4,
    "fatigue": 2, "exhausted": 2, "worn out": 2, "insomnia": 2, "oversleeping": 2, "sleep disturbances": 2,
    "eating less": 2, "eating more": 2, "appetite changes": 2,
    "low libido": 2, "disinterest in sex": 2, "sexual apathy": 2,
    "health anxiety": 1, "hypochondria": 1, "preoccupied with health": 1
}

# Function to calculate scores based on the presence and frequency of keywords
def calculate_scores(texts):
    scores = []
    for text_list in texts:
        # Convert each item in the list to string (if it's a list), then join
        text = ' '.join(str(item) for item in text_list) if isinstance(text_list, list) else str(text_list)
        words = word_tokenize(text.lower())
        score = sum(words.count(k) * v for k, v in symptom_keywords.items())
        scores.append(score)
    return np.array(scores)


# Pre-defined labels based on manual thresholding from an expert (for demonstration)
labels = np.array(combined_data['PHQ8_Binary'].tolist())

# Calculate scores and classify based on a defined threshold, e.g., 50
scores = calculate_scores(final_text)
classified_results = (scores >= 10).astype(int)  # Applying threshold to classify

# 5-Fold Stratified Cross-Validation to evaluate the effectiveness
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
accuracies, sensitivities, specificities, f1scores = [], [], [], []

for train_index, test_index in skf.split(scores, labels):
    y_train, y_test = labels[train_index], labels[test_index]
    y_pred = classified_results[test_index]

    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    spec = tn / (tn + fp) if (tn + fp) > 0 else 0

    accuracies.append(acc)
    sensitivities.append(rec)
    specificities.append(spec)
    f1scores.append(f1)

# Output average scores and their standard deviations
print("Accuracy - Mean: {:.3f}, Std: {:.3f}".format(np.mean(accuracies), np.std(accuracies)))
print("Sensitivity - Mean: {:.3f}, Std: {:.3f}".format(np.mean(sensitivities), np.std(sensitivities)))
print("Specificity - Mean: {:.3f}, Std: {:.3f}".format(np.mean(specificities), np.std(specificities)))
print("F1 Score - Mean: {:.3f}, Std: {:.3f}".format(np.mean(f1scores), np.std(f1scores)))


## **chatgpt 4o**

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix, precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
import numpy as np
import xgboost as xgb

# Define function to calculate specificity for binary classification
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp) if (tn + fp) != 0 else 0

# Example data
# final_text should be your processed text data
# labels should be your binary labels

# Assuming final_text is a list of lists containing tokens and labels is a list of binary labels

# Flatten each inner list to a single string
flattened_text = [' '.join(map(str, doc)) for doc in final_text]

# Vectorize the text data with bigrams included
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(flattened_text)
y = np.array(labels)  # Ensure y is a numpy array

kf = KFold(n_splits=5)

f1_scores = []
sensitivities = []
specificities = []
precisions = []
accuracies=[]

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Handle class imbalance using SMOTE
    smote = SMOTE()
    X_train, y_train = smote.fit_resample(X_train, y_train)

    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    acc=accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    sensitivity = recall_score(y_test, y_pred)
    specificity = specificity_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    accuracies.append(acc)
    f1_scores.append(f1)
    sensitivities.append(sensitivity)
    specificities.append(specificity)
    precisions.append(precision)

acc_mean = np.mean(accuracies)
acc_std = np.std(accuracies)
f1_mean = np.mean(f1_scores)
f1_std = np.std(f1_scores)
sensitivity_mean = np.mean(sensitivities)
sensitivity_std = np.std(sensitivities)
specificity_mean = np.mean(specificities)
specificity_std = np.std(specificities)
precision_mean = np.mean(precisions)
precision_std = np.std(precisions)

print(f"Accuracy: Mean = {acc_mean}, Std Dev = {acc_std}")
print(f"F1 Score: Mean = {f1_mean}, Std Dev = {f1_std}")
print(f"Sensitivity: Mean = {sensitivity_mean}, Std Dev = {sensitivity_std}")
print(f"Specificity: Mean = {specificity_mean}, Std Dev = {specificity_std}")
print(f"Precision: Mean = {precision_mean}, Std Dev = {precision_std}")


In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix, precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from imblearn.over_sampling import SMOTE
import numpy as np
import xgboost as xgb

# Define function to calculate specificity for binary classification
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp) if (tn + fp) != 0 else 0

# Example data
# final_text should be your processed text data
# labels should be your binary labels

# Assuming final_text is a list of lists containing tokens and labels is a list of binary labels

# Flatten each inner list to a single string
flattened_text = [' '.join(map(str, doc)) for doc in final_text]

# Vectorize the text data with bigrams included
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_text = vectorizer.fit_transform(flattened_text)

# Define the symptoms from Burn’s Depression Checklist
symptoms = [
    'Feeling sad or down in the dumps', 'Feeling unhappy or blue', 'Crying spells or tearfulness',
    'Feeling discouraged', 'Feeling hopeless', 'Low self-esteem', 'Feeling worthless or inadequate',
    'Guilt or shame', 'Criticizing yourself or blaming others', 'Difficulty making decisions',
    'Loss of interest in family, friends or colleagues', 'Loneliness', 'Spending less time with family or friends',
    'Loss of motivation', 'Loss of interest in work or other activities', 'Avoiding work or other activities',
    'Loss of pleasure or satisfaction in life', 'Feeling tired', 'Difficulty sleeping or sleeping too much',
    'Decreased or increased appetite', 'Loss of interest in sex', 'Worrying about your health',
    'Do you have any suicidal thoughts?', 'Would you like to end your life?', 'Do you have a plan for harming yourself?'
]

# Placeholder for symptom ratings
symptom_ratings = np.zeros((X_text.shape[0], len(symptoms)))

# Regression model to predict symptom ratings (for simplicity, using Linear Regression)
regressor = LinearRegression()

# Train a regression model for each symptom and predict ratings
for i, symptom in enumerate(symptoms):
    # Use some heuristic to create a target variable for training (example: keyword matching)
    # Here we use a dummy target variable, in practice you need a properly labeled dataset
    y_symptom = np.random.randint(0, 5, size=X_text.shape[0])

    regressor.fit(X_text, y_symptom)
    symptom_ratings[:, i] = regressor.predict(X_text)

# Sum the predicted ratings to get the total score
total_scores = symptom_ratings.sum(axis=1)

# Define the depression levels
depression_levels = {
    (0, 5): 'No Depression',
    (6, 10): 'Normal but unhappy',
    (11, 25): 'Mild depression',
    (26, 50): 'Moderate depression',
    (51, 75): 'Severe depression',
    (76, 100): 'Extreme depression'
}

# Convert total scores to depression levels
def get_depression_level(score):
    for (low, high), level in depression_levels.items():
        if low <= score <= high:
            return level
    return 'Unknown'

depression_labels = np.array([get_depression_level(score) for score in total_scores])

# Convert depression levels to binary labels (example: Severe and Extreme depression as positive class)
binary_labels = np.array([1 if level in ['Severe depression', 'Extreme depression'] else 0 for level in depression_labels])

# Extract the 'PHQ8_Binary' label
labels = combined_data['PHQ8_Binary'].tolist()
y = np.array(labels)  # Ensure y is a numpy array

kf = KFold(n_splits=5)

f1_scores = []
sensitivities = []
specificities = []
precisions = []

for train_index, test_index in kf.split(X_text):
    X_train, X_test = X_text[train_index], X_text[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Handle class imbalance using SMOTE
    smote = SMOTE()
    X_train, y_train = smote.fit_resample(X_train, y_train)

    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    acc=accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    sensitivity = recall_score(y_test, y_pred)
    specificity = specificity_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    accuracies.append(acc)
    f1_scores.append(f1)
    sensitivities.append(sensitivity)
    specificities.append(specificity)
    precisions.append(precision)

acc_mean = np.mean(accuracies)
acc_std = np.std(accuracies)
f1_mean = np.mean(f1_scores)
f1_std = np.std(f1_scores)
sensitivity_mean = np.mean(sensitivities)
sensitivity_std = np.std(sensitivities)
specificity_mean = np.mean(specificities)
specificity_std = np.std(specificities)
precision_mean = np.mean(precisions)
precision_std = np.std(precisions)

print(f"Accuracy: Mean = {acc_mean}, Std Dev = {acc_std}")
print(f"F1 Score: Mean = {f1_mean}, Std Dev = {f1_std}")
print(f"Sensitivity: Mean = {sensitivity_mean}, Std Dev = {sensitivity_std}")
print(f"Specificity: Mean = {specificity_mean}, Std Dev = {specificity_std}")
print(f"Precision: Mean = {precision_mean}, Std Dev = {precision_std}")



Accuracy: Mean = 0.6781887150308203, Std Dev = 0.1253890805876333
F1 Score: Mean = 0.2129718234981393, Std Dev = 0.1587420366113971
Sensitivity: Mean = 0.22326340326340324, Std Dev = 0.23895984320099856
Specificity: Mean = 0.8999444068815265, Std Dev = 0.07337868398955903
Precision: Mean = 0.38571428571428573, Std Dev = 0.3402013542760334


### **Claude 3**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import f1_score, recall_score, confusion_matrix, precision_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE

# Assuming final_text is your list of lists containing text data
# final_text = [[utterance1, utterance2, ...], [utterance1, utterance2, ...], ...]

# Extract the 'PHQ8_Binary' label
labels = combined_data['PHQ8_Binary'].tolist()

# Calculate specificity and sensitivity
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    f1 = f1_score(y_true, y_pred)
    return specificity, sensitivity, f1

# Read the labels data
labels_df = pd.read_csv('train_split_Depression_AVEC2017.csv')

# Preprocess transcript data
def preprocess_transcript(utterances):
    # Convert all elements to strings and join
    return ' '.join(str(utterance) for utterance in utterances if utterance is not None)

# Extract features from transcripts
def extract_features(texts):
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    features = vectorizer.fit_transform(texts)
    return pd.DataFrame(features.toarray(), columns=vectorizer.get_feature_names_out())

# Preprocess transcripts
processed_transcripts = [preprocess_transcript(transcript) for transcript in final_text]

# Extract features
features = extract_features(processed_transcripts)

# Prepare data for cross-validation
X = features
y = np.array(labels)  # Ensure y is a numpy array

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True)

# Initialize lists to store performance metrics
specificities = []
sensitivities = []
f1_scores = []
accuracies=[]

# Perform 5-fold cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X, y), 1):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Apply SMOTE for oversampling
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Train the model
    model = LogisticRegression(max_iter=1000, class_weight='balanced')
    model.fit(X_train_resampled, y_train_resampled)

    # Make predictions
    y_pred = model.predict(X_val_scaled)

    # Calculate metrics
    specificity, sensitivity, f1 = calculate_metrics(y_val, y_pred)
    acc=accuracy_score(y_val, y_pred)

    # Store metrics
    accuracies.append(acc)
    specificities.append(specificity)
    sensitivities.append(sensitivity)
    f1_scores.append(f1)

    print(f"Fold {fold} - Specificity: {specificity:.4f}, Sensitivity: {sensitivity:.4f}, F1-score: {f1:.4f}")

# Print average metrics
print("\nAverage metrics:")

print(f"Accuracy: {np.mean(accuracies):.4f} (+/- {np.std(accuracies):.4f})")
print(f"Specificity: {np.mean(specificities):.4f} (+/- {np.std(specificities):.4f})")
print(f"Sensitivity: {np.mean(sensitivities):.4f} (+/- {np.std(sensitivities):.4f})")
print(f"F1-score: {np.mean(f1_scores):.4f} (+/- {np.std(f1_scores):.4f})")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Improved keyword list with weights
checklist_keywords = {
    'sad': (['sad', 'down', 'unhappy', 'blue', 'cry', 'tear'], 2),
    'discouraged': (['discouraged', 'hopeless'], 2),
    'self_esteem': (['worthless', 'inadequate', 'guilt', 'shame'], 2),
    'criticizing': (['criticize', 'blame'], 1),
    'decisions': (['difficult', 'decision'], 1),
    'loss_interest': (['loss of interest', 'lonely', 'less time'], 2),
    'motivation': (['unmotivated', 'avoid work'], 2),
    'pleasure': (['no pleasure', 'no satisfaction'], 2),
    'tired': (['tired', 'fatigue'], 1),
    'sleep': (['sleep problem', 'insomnia'], 1),
    'appetite': (['appetite', 'eating'], 1),
    'sex': (['sex'], 1),
    'health': (['health', 'worry'], 1),
    'suicidal': (['suicide', 'end life', 'harm myself'], 3)
}

def estimate_weighted_ratings(text):
    ratings = []
    for category, (keywords, weight) in checklist_keywords.items():
        count = sum(text.lower().count(keyword) for keyword in keywords)
        rating = min(count * weight, 4)  # Cap at 4
        ratings.append(rating)
    return ratings

def preprocess_transcript(text_list):
    return ' '.join(str(item) for item in text_list).lower()

# Preprocess transcripts and estimate ratings
processed_transcripts = []
estimated_ratings = []
for transcript in final_text:
    processed_text = preprocess_transcript(transcript)
    processed_transcripts.append(processed_text)
    estimated_ratings.append(estimate_weighted_ratings(processed_text))

# Create a DataFrame with estimated ratings
ratings_df = pd.DataFrame(estimated_ratings, columns=checklist_keywords.keys())

# Calculate total score
ratings_df['total_score'] = ratings_df.sum(axis=1)

# Prepare data for cross-validation
X = ratings_df['total_score'].values
# Extract the 'PHQ8_Binary' label
labels = np.array(combined_data['PHQ8_Binary'].tolist())

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store performance metrics
after_accuracies = []
after_f1_scores = []
after_specificities = []
after_sensitivities = []

# Perform 5-fold cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X, y), 1):
    X_val = X[val_index]
    y_true = y[val_index]

    # Apply threshold to get predictions (using 50 as the threshold)
    y_pred = (X_val > 10).astype(int)

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)

    # Store metrics
    after_accuracies.append(accuracy)
    after_f1_scores.append(f1)
    after_specificities.append(specificity)
    after_sensitivities.append(sensitivity)

    print(f"Fold {fold}:")
    print(f"  Accuracy: {after_accuracies:}")
    print(f"  F1-score: {after_f1_scores:}")
    print(f"  Specificity: {after_specificities:}")
    print(f"  Sensitivity: {after_sensitivities:}")

# Print average metrics with standard deviations
print("\nAverage metrics:")
print(f"Accuracy: {np.mean(after_accuracies):.4f} (+/- {np.std(after_accuracies):.4f})")
print(f"F1-score: {np.mean(after_f1_scores):.4f} (+/- {np.std(after_f1_scores):.4f})")
print(f"Specificity: {np.mean(after_specificities):.4f} (+/- {np.std(after_specificities):.4f})")
print(f"Sensitivity: {np.mean(after_sensitivities):.4f} (+/- {np.std(after_sensitivities):.4f})")